In [6]:
import pandas as pd
import re
from datetime import timedelta


In [8]:
import pandas as pd

df = pd.read_csv(
    "twcs.csv",
    engine="python",
    encoding="utf-8",
    on_bad_lines="skip"
)

df.head()



,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [9]:
df.columns
df.shape

(2811774, 7)

In [10]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['clean_message'] = df['text'].apply(clean_text)


In [11]:
def classify_issue(msg):
    if "payment" in msg or "card" in msg or "charge" in msg:
        return "PAYMENT"
    elif "login" in msg or "password" in msg or "signin" in msg:
        return "LOGIN"
    elif "delivery" in msg or "delay" in msg or "late" in msg:
        return "DELIVERY"
    elif "refund" in msg or "return" in msg:
        return "REFUND"
    elif "bug" in msg or "error" in msg or "issue" in msg:
        return "BUG"
    else:
        return "GENERAL"

df['issue_type'] = df['clean_message'].apply(classify_issue)


In [12]:
df[['clean_message','issue_type']].head()


,clean_message,issue_type
0,i understand i would like to assist you we wou...,GENERAL
1,sprintcare and how do you propose we do that,GENERAL
2,sprintcare i have sent several private message...,GENERAL
3,please send us a private message so that we ca...,GENERAL
4,sprintcare i did,GENERAL


In [14]:
def assign_priority(msg):
    if "urgent" in msg or "immediately" in msg or "now" in msg:
        return "P0"
    elif "asap" in msg or "soon" in msg:
        return "P1"
    elif "help" in msg:
        return "P2"
    else:
        return "P3"

df['priority'] = df['clean_message'].apply(assign_priority)




In [15]:
df[['clean_message', 'priority']].head()

,clean_message,priority
0,i understand i would like to assist you we wou...,P3
1,sprintcare and how do you propose we do that,P3
2,sprintcare i have sent several private message...,P3
3,please send us a private message so that we ca...,P3
4,sprintcare i did,P3


In [16]:
def sla_hours(priority):
    return {
        "P0": 2,
        "P1": 6,
        "P2": 24,
        "P3": 48
    }[priority]

df['sla_hours'] = df['priority'].apply(sla_hours)


In [17]:
df[['priority', 'sla_hours']].head()


,priority,sla_hours
0,P3,48
1,P3,48
2,P3,48
3,P3,48
4,P3,48


In [18]:
import pandas as pd
from datetime import timedelta

df['created_time'] = pd.to_datetime(df['created_at'], errors='coerce')
df['due_time'] = df['created_time'] + df['sla_hours'].apply(lambda x: timedelta(hours=x))

/tmp/ipython-input-3028443097.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created_time'] = pd.to_datetime(df['created_at'], errors='coerce')


In [19]:
df[['created_time','sla_hours','due_time']].head()

,created_time,sla_hours,due_time
0,2017-10-31 22:10:47+00:00,48,2017-11-02 22:10:47+00:00
1,2017-10-31 22:11:45+00:00,48,2017-11-02 22:11:45+00:00
2,2017-10-31 22:08:27+00:00,48,2017-11-02 22:08:27+00:00
3,2017-10-31 21:54:49+00:00,48,2017-11-02 21:54:49+00:00
4,2017-10-31 21:49:35+00:00,48,2017-11-02 21:49:35+00:00


In [20]:
report = (
    df.groupby(['issue_type', 'priority'])
      .size()
      .reset_index(name='ticket_count')
)

report


,issue_type,priority,ticket_count
0,BUG,P0,22367
1,BUG,P1,2630
2,BUG,P2,29470
3,BUG,P3,110200
4,DELIVERY,P0,21063
5,DELIVERY,P1,3413
6,DELIVERY,P2,27995
7,DELIVERY,P3,98287
8,GENERAL,P0,211717
9,GENERAL,P1,38695


In [21]:
df.to_csv("final_ticket_triage_output_full.csv", index=False)